In [1]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2
import albumentations as A
from albumentations.pytorch import ToTensorV2
#from torchvision.transforms import functional as F
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import numpy as np
import os
import glob as glob
from lxml import etree, objectify
import random
from xml.etree import ElementTree as et
from datasets import (
    create_train_dataset, create_valid_dataset, 
    create_train_loader, create_valid_loader
)
from utils.transforms import (
    get_train_transform, get_valid_transform,
    get_train_aug
)

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.12 (you have 1.4.11). Upgrade using: pip install --upgrade albumentations


In [2]:
# Paths to the image and annotation directories
train_images_path = '/teamspace/studios/this_studio/football-players-detection-9/train'
train_labels_path = '/teamspace/studios/this_studio/football-players-detection-9/train'
valid_images_path = '/teamspace/studios/this_studio/football-players-detection-9/valid'
valid_labels_path = '/teamspace/studios/this_studio/football-players-detection-9/valid'
test_images_path = '/teamspace/studios/this_studio/football-players-detection-9/test'
test_labels_path = '/teamspace/studios/this_studio/football-players-detection-9/test'

# Image dimensions
width, height = 1920, 1080

# Classes (ensure these are in the same order as used during training)
classes = ['__background__', 'ball', 'goalkeeper', 'player', 'referee']

In [3]:
# Create datasets
train_dataset = create_train_dataset(
    train_images_path, train_labels_path, 
    width, height, classes
)
valid_dataset = create_valid_dataset(
    valid_images_path, valid_labels_path, 
    width, height, classes
)
test_dataset = create_valid_dataset(
    test_images_path, test_labels_path, 
    width, height, classes
)

# Create data loaders
train_loader = create_train_loader(train_dataset, batch_size=64, num_workers=4)
valid_loader = create_valid_loader(valid_dataset, batch_size=64, num_workers=4)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/albumentations/core/composition.py:156: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [4]:
augmentation_pipeline = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.1),
    #A.CLAHE(clip_limit=2, p=0.2),  # Add CLAHE for better contrast control
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.4),
    A.RandomCrop(width=450, height=450, p=0.4),
    A.GaussianBlur(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.2),
    A.OneOf([
        A.RandomRain(p=0.2),
        A.RandomSnow(p=0.2),
        A.RandomFog(p=0.2),
        A.RandomSunFlare(p=0.2)
    ], p=0.5),  # Add weather augmentation with a 30% probability
    #ToTensorV2(p=1.0)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


In [5]:
def save_augmented_image_and_labels(image, bboxes, class_labels, original_image_path, original_label_path, output_image_dir, output_label_dir, counter, class_mapping):
    image_name = os.path.splitext(os.path.basename(original_image_path))[0]
    augmented_image_name = f"{image_name}_aug_{counter}.jpg"
    augmented_label_name = f"{image_name}_aug_{counter}.xml"
    
    augmented_image_path = os.path.join(output_image_dir, augmented_image_name)
    augmented_label_path = os.path.join(output_label_dir, augmented_label_name)

    # Convert the tensor image back to a NumPy array and scale pixel values if necessary
    if torch.is_tensor(image):
        image = image.permute(1, 2, 0).cpu().numpy()

    # Ensure the image is in the range [0, 255]
    if image.max() <= 1.0:
        image = (image * 255).astype(np.uint8)
    
    cv2.imwrite(augmented_image_path, image)
    
    # Create the XML annotation
    E = objectify.ElementMaker(annotate=False)
    anno_tree = E.annotation(
        E.folder(''),
        E.filename(augmented_image_path),
        E.path(augmented_image_path),
        E.source(
            E.database("Unknown")
        ),
        E.size(
            E.width(image.shape[1]),
            E.height(image.shape[0]),
            E.depth(image.shape[2])
        ),
        E.segmented(0),
    )
    
    for bbox, class_label in zip(bboxes, class_labels):
        xmin, ymin, xmax, ymax = bbox
        class_name = class_mapping[int(class_label)]
        obj = E.object(
            E.name(class_name),
            E.pose("Unspecified"),
            E.truncated(0),
            E.difficult(0),
            E.occluded(0),
            E.bndbox(
                E.xmin(int(xmin)),
                E.ymin(int(ymin)),
                E.xmax(int(xmax)),
                E.ymax(int(ymax))
            )
        )
        anno_tree.append(obj)

    etree.ElementTree(anno_tree).write(augmented_label_path, pretty_print=True)


In [6]:
def process_augmentation(base_dir, output_dir, classes_to_augment, augmentation_pipeline, width, height, classes, class_mapping):
    for folder in ['test', 'valid', 'train']:
        image_dir = os.path.join(base_dir, folder)
        label_dir = os.path.join(base_dir, folder)
        #if folder == 'train':
         #   dataset = create_train_dataset(image_dir, label_dir, width, height, classes, use_train_aug=False, mosaic=True)
        #else:
         #   dataset = create_valid_dataset(image_dir, label_dir, width, height, classes)

        dataset = create_valid_dataset(image_dir, label_dir, width, height, classes)
        output_image_dir = os.path.join(output_dir, folder)
        output_label_dir = os.path.join(output_dir, folder)

        os.makedirs(output_image_dir, exist_ok=True)
        os.makedirs(output_label_dir, exist_ok=True)

        for image, target in dataset:
            image_path = target['image_path']
            label_path = target['annot_path']

            # Ensure the image is a NumPy array
            if not isinstance(image, np.ndarray):
                if torch.is_tensor(image):
                    image = image.permute(1, 2, 0).cpu().numpy()
                else:
                    raise TypeError("Image must be a NumPy array or a torch tensor.")

            # Convert the image to uint8 if it’s not
            if image.dtype != np.uint8:
                image = (image * 255).astype(np.uint8)

            if any(cls in target['labels'].tolist() for cls in classes_to_augment):
                for i in range(10):  # Number of augmentations per image
                    try:
                        augmented = augmentation_pipeline(image=image, bboxes=target['boxes'].tolist(), labels=target['labels'].tolist())
                        augmented_image = augmented['image']
                        augmented_bboxes = augmented['bboxes']
                        augmented_class_labels = augmented['labels']

                        # Convert augmented image to NumPy array if it's a tensor
                        if isinstance(augmented_image, torch.Tensor):
                            augmented_image = augmented_image.permute(1, 2, 0).cpu().numpy()

                        if len(augmented_bboxes) > 0 and len(augmented_class_labels) > 0:
                            save_augmented_image_and_labels(augmented_image, augmented_bboxes, augmented_class_labels, image_path, label_path, output_image_dir, output_label_dir, i, class_mapping)
                    
                    except Exception as e:
                        print(f'Error during augmentation: {e}')

    print("Augmentation complete.")

In [7]:
# Image dimensions
width, height = 1920, 1080

# Classes (ensure these are in the same order as used during training)
classes = ['__background__', 'ball', 'goalkeeper', 'player', 'referee']

base_dir = '/teamspace/studios/this_studio/football-players-detection-9'
output_dir = '/teamspace/studios/this_studio/football-players-detection-9-augmented'
classes_to_augment = [1, 2, 3, 4]
class_mapping = {
    1: "ball",
    2: "goalkeeper",
    3: "player", 
    4: "referee"
}
process_augmentation(base_dir, output_dir, classes_to_augment, augmentation_pipeline, width, height, classes, class_mapping)

Augmentation complete.
